In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

### Постановка задачи

К данной задаче прилагается 3 файла:

- **agents2019.csv –** файл содержит информацию о просрочке дебиторской задолженности (далее - ПДЗ) за 2019 год и данных, известных на момент начала 2019 года
- **agents2020.csv –** файл содержит информацию о просрочке дебиторской задолженности (ПДЗ) за 2020 год и данных, известных на момент начала 2020 года
- **agents2021.csv –** файл содержит информацию о просрочке дебиторской задолженности (ПДЗ) за 2021 год и данных, известных на момент начала 2021 года

#### Agents 2019

Самый важный(целевой) признак – **Макс.ПДЗ за 2019 год,** представляющий собой данные о максимальном сроке дебиторской задолженности на момент окончания этого года, выраженный в днях.  Анализ этого признака и возможность его предсказания являются краеугольными в данной задаче.

#### Agents 2020

Самый важный(целевой) признак – **Макс.ПДЗ за 2020 год,** представляющий собой данные о максимальном сроке дебиторской задолженности на момент окончания этого года, выраженный в днях.  Анализ этого признака и возможность его предсказания являются краеугольными в данной задаче.

#### Agents 2021

Самый важные(целевые) признаки:

---

**ПДЗ 1-30 –** отражает, была ли просрочка в течении года в диапазоне от 1 до 30 дней

**ПДЗ 31-90 -** отражает, была ли просрочка в течении года в диапазоне от 31 до 90 дней

**ПДЗ 91-365 -** отражает, была ли просрочка в течении года в диапазоне от 91 до 365 дней

**ПДЗ более 365 -**  отражает, была ли просрочка более 365 дней

#### Подробнее описано здесь

https://russianhackers.notion.site/89cabcf01f8147a9b37efd78433d0874

### Исходные данные

In [40]:
data19=pd.read_csv('agents2019.csv')
print(data19.shape)

(531, 55)


In [3]:
data20=pd.read_csv('agents2020.csv')
print(data20.shape)

(531, 131)


In [16]:
data21=pd.read_csv('agents2021.csv')
print(data21.shape)

(325, 136)


### Чистка данных

In [52]:
feats_2019=[x for x in data19.columns if x.split(',')[0] in ('2018','2017','2016') and 'ПДЗ' not in x and 'RUB' in x]
feats_2020=[x for x in data20.columns if x.split(',')[0] in ('2019','2018','2017') and 'ПДЗ' not in x and 'RUB' in x]
feats_2021=[x for x in data21.columns if x.split(',')[0] in ('2020','2019','2018') and 'ПДЗ' not in x and 'RUB' in x]

print(f'Число признаков для 2019 года: {len(feats_2019)}')
print(f'Число признаков для 2020 года: {len(feats_2020)}')
print(f'Число признаков для 2021 года: {len(feats_2021)}')
print('\n')
print(feats_2019)

Число признаков для 2019 года: 48
Число признаков для 2020 года: 48
Число признаков для 2021 года: 48


['2016, Нематериальные активы, RUB', '2017, Нематериальные активы, RUB', '2018, Нематериальные активы, RUB', '2016, Основные средства , RUB', '2017, Основные средства , RUB', '2018, Основные средства , RUB', '2016, Внеоборотные активы, RUB', '2017, Внеоборотные активы, RUB', '2018, Внеоборотные активы, RUB', '2016, Дебиторская задолженность, RUB', '2017, Дебиторская задолженность, RUB', '2018, Дебиторская задолженность, RUB', '2016, Оборотные активы, RUB', '2017, Оборотные активы, RUB', '2018, Оборотные активы, RUB', '2016, Уставный капитал , RUB', '2017, Уставный капитал , RUB', '2018, Уставный капитал , RUB', '2016, Капитал и резервы, RUB', '2017, Капитал и резервы, RUB', '2018, Капитал и резервы, RUB', '2016, Заёмные средства (долгосрочные), RUB', '2017, Заёмные средства (долгосрочные), RUB', '2018, Заёмные средства (долгосрочные), RUB', '2016, Долгосрочные обязательства, RUB', '2

#### Создаём таргеты

In [54]:
trg_2019=[x for x in data19.columns if '2019' in x and 'Макс. ПДЗ' in x][0]
trg_2020=[x for x in data20.columns if '2020' in x and 'Макс. ПДЗ' in x][0]

#### Делаем новые датасеты

In [55]:
data_2019_clean=pd.concat([data19['Наименование ДП'],data19[feats_2019],data19[trg_2019]],axis=1) # Обучающая выборка
data_2020_clean=pd.concat([data20['Наименование ДП'],data20[feats_2020],data20[trg_2020]],axis=1) # Тестовая выборка
data_2021_clean=pd.concat([data21['Наименование ДП'],data21[feats_2021]],axis=1)                  # Выборка для прогноза

#### Убираем единицы измерения

In [56]:
feats_2019_new=[x.split(', RUB')[0] for x in feats_2019]
feats_2020_new=[x.split(', RUB')[0] for x in feats_2020]
feats_2021_new=[x.split(', RUB')[0] for x in feats_2021]

for i in range(len(feats_2019)):
    data_2019_clean=data_2019_clean.rename({feats_2019[i]: feats_2019_new[i]},axis=1)
    data_2020_clean=data_2020_clean.rename({feats_2020[i]: feats_2020_new[i]},axis=1)
    data_2021_clean=data_2021_clean.rename({feats_2021[i]: feats_2021_new[i]},axis=1)

#### Сохраняем в csv

In [57]:
data_2019_clean.to_csv('df2019.csv')
data_2020_clean.to_csv('df2020.csv')
data_2021_clean.to_csv('df2021.csv')